# How to Aggregate Image-Segmentation tasks via Crowdsourcing

Welcome to this tutorial!

Aggregation of Image-Segmentation tasks is the same integral thing as text aggregation, etc. In this notebook, we will find out how to aggregate such data with methods from Crowd-Kit library by Toloka!

## Libraries importing

First of all, let's install and import necessary libraries

In [ ]:
%%capture
%pip install crowd-kit
import pandas as pd
import numpy as np
from crowdkit.datasets import load_dataset
from crowdkit.aggregation import SegmentationEM
from crowdkit.aggregation import SegmentationMajorityVote
from crowdkit.aggregation import SegmentationRASA

## Data preparation

For instance, we are going to use the *mscoco_small* data from Crowd-Kit

In [ ]:
df, gt = load_dataset('mscoco_small')

Let's see the structure of our dataset

In [ ]:
df.head()

,task,worker,segmentation
90,10,0e8e2270aac26ac915d565882e0a83db,"[[False, False, False, False, False, False, Fa..."
91,10,48b03f3a39a32c4e0a186b772b415998,"[[False, False, False, False, False, False, Fa..."
92,10,ba9b643a79bb5e5cb2eb083dc2a5ab26,"[[False, False, False, False, False, False, Fa..."
93,10,61006a693705637714e8415470b42e30,"[[False, False, False, False, False, False, Fa..."
94,10,c1b20bca2b8480af13ff8917b01d7edd,"[[False, False, False, False, False, False, Fa..."


## Aggreagtion methods

Now let's see how we can solve the aggregation problem with image segmentation. We will use three methods - *SegmentationEM*, *SegmentationMajorityVote*, and *SegmentationRASA*. All these methods solve the same problem - image segmentation aggregation - but in different ways.

## SegmentationEM

Let's use the EM algorithm for the image segmentation task.

This method performs a categorical aggregation task for each pixel: should it be included to the resulting aggregate or no. This task is solved by the single coin Dawid-Skene algorithm. Each worker has a latent parameter "skill" that shows the probability of this worker to answer correctly. Skills and true pixels' labels are optimized by the Expectation-Maximization algorithm.

In [ ]:
result_em = SegmentationEM().fit_predict(df)

Now, we need to convert our results to numpy array for convenience

In [ ]:
result_em = result_em.to_numpy()
gt = gt.to_numpy()

We will use the *IOU* metric to calculate the efficiency of the methods

In [ ]:
def calculate_iou(target, result):
    iou = np.array([])
    for i in range(0, len(result) - 1):
        overlap = result[i] * target[i] 
        union = result[i] + target[i] 
    iou = np.append(iou, overlap.sum() / float(union.sum()))
    return iou.mean()

In [ ]:
print(calculate_iou(gt, result_em))

0.8551840979658429


## Segmentation Majority Vote

The next method - Segmentation Majority Vote - chooses a pixel if more than half of workers voted.

This method implements a straightforward approach to the image segmentations aggregation: it assumes that if pixel is not inside in the worker's segmentation, this vote counts as 0, otherwise, as 1. Next, the SegmentationEM aggregates these categorical values for each pixel by the Majority Vote.

In [ ]:
result_majorityvote = SegmentationMajorityVote().fit_predict(df)
result_majorityvote = result_majorityvote.to_numpy()

In [ ]:
print(calculate_iou(gt, result_majorityvote))

0.8352633625644315


## Segmentation RASA

The last method - Segmentation RASA - chooses a pixel if sum of weighted votes of each workers' more than 0.5.

Algorithm works iteratively, at each step, the workers are reweighted in proportion to their distances to the current answer estimation. The distance is considered as 1−IOU. Modification of the RASA method for texts.

In [ ]:
result_rasa = SegmentationRASA().fit_predict(df)
result_rasa = result_rasa.to_numpy()

In [ ]:
print(calculate_iou(gt, result_rasa))

0.8395562415748342


## Summary

All the results are quite good but the SegmentationEM shows slightly better result.
In this tutorial, we looked at how to deal with the image segmentation aggregation task by using Crowd-Kit library.